##### 第一模块(导入包模块：必须运行）

In [1]:
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

from tkinter import *
from tkinter import filedialog
## tk文件导入模块

##### 第二模块(文件处理模块：必须运行）

In [2]:
def import_table(root):
    """
    return table:选中的表格
    """
    try:
        root.filename = filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.destroy()
            return table
    except Exception as e:
        root.destroy()
        print("导入错误")

def remote_select():
    print("以下文件可以调用，需要分析哪个文件？")
    print("-------------------------------------")
    index = 0
    list_item_temp = []
    for item in os.listdir():
        if ".xlsx" in item:
            index += 1
            print("[" + str(index) + "] " + item)
            list_item_temp.append(item)
    try:
        bash_pos = "/Users/dfuser/Desktop/目标文书目录/"
        file_code = int(input("需要导入哪个文件？(输入[]中的序号)"))
        final_pos = bash_pos + str(list_item_temp[file_code-1])
        table_ = pd.read_excel(final_pos)
        return table_
        print("表格导入成功，以下是表格预览")
        print("----------------------------")
    except Exception as e:
        print("导入错误")        

def save_model(table):
    """
    Param table:需要导出的表
    """
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该摘要表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该词频表的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx" #导出excel文件
            table.to_excel(save_file_name_xlsx)
        else:
            pass
    except Exception as e:
        pass

##### 第三模块(核心分类器模块：必须运行）

In [3]:
# hankcs的说明：https://github.com/hankcs/HanLP/wiki/%E6%96%87%E6%9C%AC%E8%81%9A%E7%B1%BB
def HanLp_Cluster(corpus):
    ClusterAnalyzer = SafeJClass("com.hankcs.hanlp.mining.cluster.ClusterAnalyzer")
    analyzer = ClusterAnalyzer()
    for i,x in enumerate(corpus):
        try:
            analyzer.addDocument(i,x)
        except Exception as e:
            analyzer.addDocument(i,"None")
    # 全部转化成list对象
    table_corpus = pd.DataFrame(corpus)
    nclusters = input("输入簇心数量（若不输入或输入1自动判断聚类个数）")
    if nclusters == "":
        nclusters = 1.0
    indexes = list(map(lambda x:list(x),analyzer.repeatedBisection(nclusters)))
    classification = 1
    list_table_combination = []
    for index_num in range(0,len(indexes)):
        table_sub = table_corpus.loc[indexes[index_num]]
        table_sub["分类"] = classification
        classification += 1
        list_table_combination.append(table_sub)
    classfication_result = pd.concat(list_table_combination)
    return classfication_result

#### 控制部分

In [ ]:
if __name__ == '__main__':
    # ------ 导入函数开始
    flag_input = int(input("你是在本机上操作？还是远程操作？(1:本机操作,2:远程操作)"))
    if flag_input == 1:
        root = Tk() # 实例化TKinter窗口
        root.withdraw() # 隐藏TKinter窗口
        table_ready_to_eat = import_table(root)
    elif flag_input == 2:
        table_ready_to_eat = remote_select()
    else:
        logging.error("加载错误")
    # ------ 导入函数结束
    
    print("表格中所有的列:",table_ready_to_eat.columns)
    selected_column = input("请问需要分析哪一列的关键词词频？(输入关键词部分):")
    corpus = table_ready_to_eat[selected_column]
    classfication_result = HanLp_Cluster(table_ready_to_eat[selected_column])
    save_model(classfication_result)

你是在本机上操作？还是远程操作？(1:本机操作,2:远程操作)2
以下文件可以调用，需要分析哪个文件？
-------------------------------------
[1] 夫妻债务 - 原表.xlsx
[2] wordfreqs.xlsx
[3] ex1.xlsx
[4] 校园暴力 - 结构化重构.xlsx
[5] 校园暴力事件 - 原表.xlsx
[6] 正当防卫 - 原表.xlsx
[7] 夫妻债务.xlsx
[8] .~zhaiwu_regex_filter.xlsx.xlsx
[9] 职业打假人 - 原表.xlsx
需要导入哪个文件？(输入[]中的序号)1
表格中所有的列: Index(['标题', '案号', '案件类型', '庭审程序', '案由', '文书类型', '法院', '判决日期', '原告', '被告',
       '第三人', '法官', '审判长', '审判员', '书记员', '头部', '头部2', '当事人', '当事人2', '庭审程序说明',
       '庭审程序说明2', '庭审过程', '庭审过程2', '庭审过程3', '庭审过程4', '庭审过程5', '庭审过程6', '法院意见',
       '法院意见2', '判决结果', '判决结果2', '庭后告知', '庭后告知2', '结尾', '结尾2', '附录', '附录2'],
      dtype='object')
请问需要分析哪一列的关键词词频？(输入关键词部分):法院意见


In [5]:
classfication_result

,判决结果,分类
1152,NaN,1
3,NaN,1
899,NaN,1
5,NaN,1
263,NaN,1
776,NaN,1
137,NaN,1
905,NaN,1
778,NaN,1
1034,NaN,1
